## Import TensforFlow and Numpy, make sure the package is there

In [ ]:
import numpy as np
import tensorflow as tf

# Read the text into a list of strings, using tensorflow

In [ ]:
import zipfile as zf

#This function return a list of words
def read_data(filename):
  with zf.ZipFile(filename) as f:
    raw=f.read(f.namelist()[0])
    data = tf.compat.as_str(raw).split()    
        #tf.compat is not in online document
  return data

word_list = read_data('text/text8.zip')
#data size is the total number of words in the file
print 'Data size: ', len(word_list)

# Build dataset

In [ ]:
import collections

#This is our universe, we don't study any word outside this vocabulary 
vocabulary_size = 50000  

#Replace every word by its index, this is our dataset; replace rare words with token "UNK"
def build_dataset(words):
  #initialize an array with word and its count, with the first word as 'UNK'
  word_Count = [['UNK', -1]]   
  #Find common words, and add them to our vocabulary array
  unique_words=collections.Counter(word_list)
  print "unique words: ", len(unique_words)
  # We are using a function from "Collection", most_common  
  common_words=unique_words.most_common(vocabulary_size-1)  
          #make sure this is "vocabulary_size-1" as we already have 1 word in 
  word_Count.extend(common_words)
  """After the above operation, word_Count is a 2-d array with 50,001 words and their count"""

  #Initialize an empty hashtable
  dictionary = dict()
  for word, _ in word_Count:
    dictionary[word] = len(dictionary)  #current size, thus it is an index
  """After the above operation, dictionary is a hashtable with 50,001 words and their index, ordered by Count"""

  #Create our dataset, where every word is replaced by its index
  data = list()
  unk_count = 0
  for word in word_list:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
    
  #Update the count for UNK, any word that is not common is considered as UNK
  word_Count[0][1] = unk_count  #this is a side-product

  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) #zip just make the hashtable more compact
    
  return data, word_Count, dictionary, reverse_dictionary

#It takes about 4 seconds to finish
data, word_Count, dictionary, reverse_dictionary = build_dataset(word_list)
del word_list  # to reduce memory.

print('top 10 common words (+UNK)', word_Count[:10])   #the top 10 common words
print ('data= ', data[:10])
print('Sample data',  [reverse_dictionary[i] for i in data[:10]])

# Generate a training batch for the skip-gram model.
# Reference paper:

In [ ]:
import random

data_index = 0

batch_size = 128  #the number of words we process each time 
embedding_size = 128  # Dimension of the embedding vector.
num_skips = 2      # How many times to reuse an input to generate a label.
skip_window = 1      # Number of words to consider left and right.

#Given batch size, generate labeled data from original sequence
#for every word, find its label, which is within the skip-window
def generate_batch(batch_size, num_skips, skip_window):
  global data_index

  assert batch_size % num_skips == 0  #if this condition does not hold, the system will issue error
  assert num_skips <= 2 * skip_window

  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)

  """Read data if size span into buffer """ 
  ## Span is the total number of words, before + after+ current word
  span = 2 * skip_window + 1   #span=3
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = data_index + 1
  
  """ The actual number of operation is batch_size // num_skips, where "//" is floor division, removing decimal numbers """ 
  for i in range(batch_size // num_skips):
    # start with the center word, 
    target = skip_window  
    
    # this is a list that records what we have processed
    targets_to_avoid = [ skip_window ]   
    
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = data_index + 1
    
  return batch, labels

#Some sample output
batch, labels = generate_batch(batch_size=10, num_skips=2, skip_window=1)
print batch
print labels
for i in range(len(batch)):
  print(reverse_dictionary[batch[i]], '->', reverse_dictionary[labels[i, 0]])
print "data index=",data_index

# Build and train a skip-gram model.

In [ ]:
#Some sample output
batch, labels = generate_batch(batch_size=10, num_skips=2, skip_window=1)
print batch
print labels
for i in range(len(batch)):
  print(reverse_dictionary[batch[i]], '->', reverse_dictionary[labels[i, 0]])
print "data index=",data_index

In [ ]:
import math

# We pick a random validation set to sample nearest neighbors. Here we limit the validation samples to frequent words .
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()
with graph.as_default():
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
     #a list if integers that are indexes for words
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
     #a list if integers that are indexes for words
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    #word vectors are all intialized as numbers between -1 and 1 
    wordsE = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    
    #inputE is a smaller size list than all wordsE, 
    inputE = tf.nn.embedding_lookup(wordsE, train_inputs)
    #The resulting dimension is [batch_size, embedding_size]
    
    # Construct the variables
    #The weights is initialized from a normal distribution, with mean 0 and standard deviation 1/128^2, almost 0
    initial_weights=tf.truncated_normal([vocabulary_size, embedding_size], stddev=1.0 / math.sqrt(embedding_size))
    weights = tf.Variable(initial_weights)
    biases = tf.Variable(tf.zeros([vocabulary_size]))

  loss = tf.reduce_mean(
              tf.nn.sampled_softmax_loss(weights, biases, inputE, train_labels, num_sampled, vocabulary_size))
  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(wordsE), 1, keep_dims=True))
  normalized_wordsE = wordsE / norm
    
  #calculate pairwise similarity score for every word in valid set, and all the words in vocabulary
  validE = tf.nn.embedding_lookup(normalized_wordsE, valid_dataset)
  similarity = tf.matmul(validE, normalized_wordsE, transpose_b=True)

  # Add variable initializer.
  init = tf.initialize_all_variables()


In [ ]:
with tf.Session() as sess:
    allwords =tf.Variable(tf.random_uniform([5, 2], -1.0, 1.0)) 
    #Variable has to initialized before running
    #sess.run(allwords.initializer)
    tf.initialize_all_variables().run() #this is more general
    print allwords.eval()
    batch=[1,3,4]
    bwords=tf.nn.embedding_lookup(allwords, batch)
    print bwords.eval()

# Begin training.

In [ ]:
print normalized_wordsE

In [ ]:
from six.moves import xrange   

num_steps = 100001

with tf.Session(graph=graph) as session:
  init.run()  #initialize the graph
  print("Initialized")
  #print normalized_wordsE.eval()

  data_index=0 #Reinitialize, important to do this
  average_loss = 0
  for step in xrange(num_steps):
    binputs, blabels = generate_batch(batch_size, num_skips, skip_window)
    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    assignment={train_inputs : binputs, train_labels : blabels}
    _, loss_val = session.run([optimizer, loss], feed_dict = assignment )
    average_loss += loss_val

    if step % 2000 == 0:
      # The average loss is an estimate of the loss over the last 2000 batches.
      if step > 0:
        average_loss /= 2000
      print("Average loss at step ", step, ": ", average_loss)
      average_loss = 0
    
    """
    if step == 2000:
      print binputs  #a list of indexes
      #print blabels
      print wordsE.eval()  
      print normalized_wordsE.eval()
      print inputE
      #print inputE.eval()
    """
    
    # Note that this is expensive, so only doing it every 10,000
    if step % 10000 == 0:  
      sim = similarity.eval()  #get pairwise similarity for every word
      for i in range(5):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log_str = "Nearest to %s:" % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = "%s %s," % (log_str, close_word)
        print(log_str)
        
  final_wordsE = normalized_wordsE.eval()


 # Visualize the results

In [ ]:
print(final_wordsE)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
from sklearn.manifold import TSNE

In [ ]:
def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
  assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
  plt.figure(figsize=(18, 18))  #in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i,:]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')
  plt.savefig(filename)

tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
plot_only = 500
low_dim_embs = tsne.fit_transform(final_wordsE[:plot_only,:])
labels = [reverse_dictionary[i] for i in xrange(plot_only)]
plot_with_labels(low_dim_embs, labels)
